In [1]:
CIFAR = '/content/drive/My Drive/Assignment/ML ASS 3/cifar-10-batches-py/'
import numpy as np

In [2]:
def unpickle(file):
    import pickle
    with open(file,'rb') as fo:
        cifar_dict = pickle.load(fo,encoding='bytes')
    return cifar_dict

In [3]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data = [0,1,2,3,4,5,6]

for i, direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR+direc)

In [4]:
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]

In [5]:
all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
test_batch1 = [test_batch]

training_image = np.vstack([d[b"data"] for d in all_train_batches])
train_len = len(training_image)
training_image = training_image.reshape(train_len,3*32*32)
training_label = np.hstack([d[b"labels"] for d in all_train_batches])

In [6]:
test_image = np.vstack([d[b"data"] for d in test_batch1])
test_len = len(test_image)
test_image = test_image.reshape(test_len,3*32*32)
test_label = np.hstack([d[b"labels"] for d in test_batch1])

In [7]:
##### RAJATA CODE for hog + color hist
import numpy as np
import matplotlib.colors as colors
from skimage.feature import hog
from skimage import color

X_train_hog = []

for img in training_image:

    img = np.reshape(img.T,(32,32,3))
    # color histogram
    array=np.asarray(img)
    arr=(array.astype(float))/255.0
    img_hsv = colors.rgb_to_hsv(arr[...,:3])
    img_color_hist = np.histogram(img_hsv[...,0],bins=10)
    hist_values = img_color_hist[0]

    #hog
    gray_image = color.rgb2gray(img)
    hog_feature = hog(gray_image,orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1))

    #joined
    X_train_hog.append(np.concatenate((hist_values,hog_feature),axis=None))

In [8]:
X_test_hog = []

for img in test_image:

    img = np.reshape(img.T,(32,32,3))
    # color histogram
    array=np.asarray(img)
    arr=(array.astype(float))/255.0
    img_hsv = colors.rgb_to_hsv(arr[...,:3])
    img_color_hist = np.histogram(img_hsv[...,0],bins=10)
    hist_values = img_color_hist[0]

    #hog
    gray_image = color.rgb2gray(img)
    hog_feature = hog(gray_image,orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1))

    #joined
    X_test_hog.append(np.concatenate((hist_values,hog_feature),axis=None))

In [9]:
import numpy
X_train =numpy.asarray(X_train_hog)
X_test = numpy.asarray(X_test_hog)

In [10]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC 
import pickle 
# defining parameter range 
param_grid = {'C': [0.1],                 
              'kernel': ['linear']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1) 
  
# fitting the model for grid search 
grid.fit(X_train, training_label) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 144.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1], 'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [11]:
print(grid.best_params_) 

{'C': 0.1, 'kernel': 'linear'}


In [16]:
sv = grid.best_estimator_.fit(X_train,training_label)

In [17]:
filename = 'SVMlinear01 HOG .sav'
pickle.dump(sv, open(filename, 'wb'))

In [12]:
grid_trainpredict = grid.predict(X_train)

In [13]:
grid_predictions = grid.predict(X_test) 

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(training_label, grid_trainpredict))

              precision    recall  f1-score   support

           0       0.47      0.42      0.44      5000
           1       0.37      0.46      0.41      5000
           2       0.30      0.21      0.25      5000
           3       0.29      0.17      0.21      5000
           4       0.34      0.33      0.34      5000
           5       0.32      0.46      0.38      5000
           6       0.38      0.55      0.45      5000
           7       0.32      0.31      0.32      5000
           8       0.45      0.44      0.45      5000
           9       0.32      0.26      0.29      5000

    accuracy                           0.36     50000
   macro avg       0.36      0.36      0.35     50000
weighted avg       0.36      0.36      0.35     50000



In [15]:
print(classification_report(test_label, grid_predictions))

              precision    recall  f1-score   support

           0       0.47      0.41      0.44      1000
           1       0.36      0.43      0.39      1000
           2       0.29      0.20      0.24      1000
           3       0.23      0.13      0.17      1000
           4       0.32      0.31      0.32      1000
           5       0.31      0.46      0.37      1000
           6       0.37      0.53      0.44      1000
           7       0.32      0.31      0.31      1000
           8       0.45      0.43      0.44      1000
           9       0.30      0.25      0.27      1000

    accuracy                           0.35     10000
   macro avg       0.34      0.35      0.34     10000
weighted avg       0.34      0.35      0.34     10000

